In [1]:
import requests
import pandas as pd
import json
import numpy as np
import matplotlib.pyplot as plt

apiKey = "gQvOAvCu725jH4VcgCA300hbQJNwFfz2"
apiUrl = "https://aeroapi.flightaware.com/aeroapi/"

operators = "UAL"
payload = {'max_pages': 3}
auth_header = {'x-apikey':apiKey}

response = requests.get(apiUrl + f"operators/{operators}/flights",
    params=payload, headers=auth_header)

if response.status_code == 200:
    output = response.json()
else:
    print("Error executing request")




In [2]:
file_path = "C:/Users/FJL767/Desktop/Flight delay modelling/FlightAware_UAL.json"
with open(file_path, 'w') as f:
    json.dump(output, f)

In [30]:
'''apiKey = "gQvOAvCu725jH4VcgCA300hbQJNwFfz2"
apiUrl = "https://aeroapi.flightaware.com/aeroapi/"

airports = 'SFO'
payload = {'max_pages': 2}
auth_header = {'x-apikey':apiKey}

response = requests.get(apiUrl + f"airports/{airports}/flights",
    params=payload, headers=auth_header)

if response.status_code == 200:
    airport_output = response.json()
else:
    print("Error executing request")

file_path = "C:/Users/FJL767/Desktop/Flight delay modelling/FlightAware_KSFO.json"
with open(file_path, 'w') as f:
    json.dump(airport_output, f)'''

'apiKey = "gQvOAvCu725jH4VcgCA300hbQJNwFfz2"\napiUrl = "https://aeroapi.flightaware.com/aeroapi/"\n\nairports = \'SFO\'\npayload = {\'max_pages\': 2}\nauth_header = {\'x-apikey\':apiKey}\n\nresponse = requests.get(apiUrl + f"airports/{airports}/flights",\n    params=payload, headers=auth_header)\n\nif response.status_code == 200:\n    airport_output = response.json()\nelse:\n    print("Error executing request")\n\nfile_path = "C:/Users/FJL767/Desktop/Flight delay modelling/FlightAware_KSFO.json"\nwith open(file_path, \'w\') as f:\n    json.dump(airport_output, f)'

In [3]:
df = pd.json_normalize(output)
print(df)

                                           scheduled  \
0  [{'ident': 'UAL855', 'ident_icao': 'UAL855', '...   

                                            arrivals  \
0  [{'ident': 'UAL297', 'ident_icao': 'UAL297', '...   

                                             enroute  num_pages  \
0  [{'ident': 'UAL6657', 'ident_icao': 'UAL6657',...          3   

                                          links.next  
0  /operators/UAL/flights?max_pages=3&cursor=ef57...  


In [4]:
"""arrivals = pd.json_normalize(output['arrivals'])
departures = pd.json_normalize(output['departures'])
scheduled_arrivals = pd.json_normalize(output['scheduled_arrivals'])
scheduled_departures = pd.json_normalize(output['scheduled_departures'])
print(arrivals.head())"""

scheduled = pd.json_normalize(output['scheduled'])
arrivals = pd.json_normalize(output['arrivals'])
scheduled.to_csv("C:/Users/FJL767/Desktop/Flight delay modelling/united.csv")
arrivals.to_csv("C:/Users/FJL767/Desktop/Flight delay modelling/united_2.csv")

In [5]:
arrivals_cleaned=arrivals[["ident_icao","operator","departure_delay","arrival_delay","aircraft_type","route_distance","origin.code_icao","destination.code_icao"]]
print(arrivals_cleaned)


   ident_icao operator  departure_delay  arrival_delay aircraft_type  \
0      UAL297      UAL                0           -420          B38M   
1     UAL1597      UAL             -480          -1440          B739   
2      UAL110      UAL             2220           1980          B763   
3      UAL884      UAL             -360          -2340          B764   
4      UAL946      UAL             -240           -540          B788   
5     UAL2238      UAL                0          -2040          B752   
6      UAL134      UAL             2160            -60          B763   
7     UAL1062      UAL             3120           3480          B739   
8      UAL999      UAL                0          -1620          B78X   
9       UAL14      UAL             -180          -1020          B763   
10    UAL2034      UAL             -300          -1200          A319   
11     UAL918      UAL               60           -120          B772   
12    UAL2681      UAL             -600          -1740          

In [6]:
X = arrivals_cleaned[["operator","departure_delay","aircraft_type","route_distance","origin.code_icao","destination.code_icao"]]
y = arrivals_cleaned['arrival_delay']

In [7]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(X['operator'])
sequences = tokenizer.texts_to_sequences(X['operator'])
max_sequence_length = 20
padded_sequences = pad_sequences(sequences, maxlen=max_sequence_length, padding='post', truncating='post')
sequences = np.array(sequences)
X['operator'] = sequences
tokenizer.fit_on_texts(X['aircraft_type'])
sequences = tokenizer.texts_to_sequences(X['aircraft_type'])
max_sequence_length = 20
padded_sequences = pad_sequences(sequences, maxlen=max_sequence_length, padding='post', truncating='post')
sequences = np.array(sequences)
X['aircraft_type'] = sequences
tokenizer.fit_on_texts(X['origin.code_icao'])
sequences = tokenizer.texts_to_sequences(X['origin.code_icao'])
max_sequence_length = 20
padded_sequences = pad_sequences(sequences, maxlen=max_sequence_length, padding='post', truncating='post')
sequences = np.array(sequences)
X['origin.code_icao'] = sequences
tokenizer.fit_on_texts(X['destination.code_icao'])
sequences = tokenizer.texts_to_sequences(X['destination.code_icao'])
max_sequence_length = 20
padded_sequences = pad_sequences(sequences, maxlen=max_sequence_length, padding='post', truncating='post')
sequences = np.array(sequences)
X['destination.code_icao'] = sequences





C:\Users\FJL767\AppData\Local\Temp\ipykernel_22316\501104791.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['operator'] = sequences
C:\Users\FJL767\AppData\Local\Temp\ipykernel_22316\501104791.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['aircraft_type'] = sequences
C:\Users\FJL767\AppData\Local\Temp\ipykernel_22316\501104791.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cav

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
model = Sequential()
model.add(Dense(units=64, activation='relu', input_dim=6))  # Input layer with 6 features
model.add(Dense(units=32, activation='relu'))
model.add(Dense(units=1, activation='linear'))  # Output layer for regression
model.compile(optimizer='adam', loss='mean_squared_error')  # Use an appropriate loss function for your task
model.fit(X_train, y_train, epochs=1000, batch_size=32, validation_data=(X_test, y_test))
loss = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss}")
y_pred = model.predict(X_test)




Epoch 1/1000
2/2 [==============================] - 1s 174ms/step - loss: 22331538.0000 - val_loss: 12920302.0000
Epoch 2/1000
2/2 [==============================] - 0s 30ms/step - loss: 22331076.0000 - val_loss: 12920195.0000
Epoch 3/1000
2/2 [==============================] - 0s 32ms/step - loss: 22330672.0000 - val_loss: 12920160.0000
Epoch 4/1000
2/2 [==============================] - 0s 59ms/step - loss: 22330258.0000 - val_loss: 12920140.0000
Epoch 5/1000
2/2 [==============================] - 0s 32ms/step - loss: 22329836.0000 - val_loss: 12920135.0000
Epoch 6/1000
2/2 [==============================] - 0s 31ms/step - loss: 22329444.0000 - val_loss: 12920132.0000
Epoch 7/1000
2/2 [==============================] - 0s 23ms/step - loss: 22329032.0000 - val_loss: 12920132.0000
Epoch 8/1000
2/2 [==============================] - 0s 29ms/step - loss: 22328620.0000 - val_loss: 12920158.0000
Epoch 9/1000
2/2 [==============================] - 0s 27ms/step - loss: 22328216.0000 - val_lo

In [10]:
print(X)

    operator  departure_delay  aircraft_type  route_distance  \
0          1                0              9             447   
1          1             -480              5             338   
2          1             2220              3            3459   
3          1             -360              2            4499   
4          1             -240             12            3861   
5          1                0              6            1634   
6          1             2160              3            3939   
7          1             3120              5            1512   
8          1                0             13            3675   
9          1             -180              3            3459   
10         1             -300             14             262   
11         1               60              7            3671   
12         1             -600              8             192   
13         1             -120              4            1658   
14         1             -300           

In [11]:
print(y_pred)

[[-1785.5072 ]
 [ -466.95422]
 [-3940.4165 ]
 [ 5333.516  ]
 [ -306.56427]
 [ 2128.8638 ]
 [   40.71898]
 [-1763.4196 ]
 [  169.05515]]


In [12]:
print(y_test)

39     420
25    -960
26   -2220
43    9720
35    -900
41    3000
4     -540
12   -1740
8    -1620
Name: arrival_delay, dtype: int64


In [5]:
import tensorflow as tf
print(f"Num GPUs Available: {len(tf.config.experimental.list_physical_devices('GPU'))}")


Num GPUs Available: 0
